In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import re
import networkx as nx
import numpy as np
import seaborn as sns
from flask import Flask, render_template_string
import matplotlib.pyplot as plt
import io
import base64
from torch import embedding
!pip install flask-ngrok
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, request
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster


In [ ]:
!curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
!chmod +x ./ngrok
!./ngrok authtoken 2YzojbvlvYkzVrjmqyym1IkjxbN_rs4jcvbEkQ7Nfi5jbCiD

from google.colab import drive
drive.mount('/content/drive')

In [ ]:


  app = Flask(__name__)
  run_with_ngrok(app)  # Start ngrok when the app is run

  @app.route('/')
  def display_output():



      # Load the dataset
      df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/collection_jobs.csv', delimiter='|', error_bad_lines=False)

      # Preprocess and clean text data
      df['title'] = df['title'].str.lower()  # Lowercase
      df['description'] = df['description'].str.lower()  # Lowercase

      # Remove rows with all whitespace in all columns
      df = df.dropna(how='all', subset=df.columns[1:], inplace=False)

      # Define a regular expression pattern to match timestamps
      timestamp_pattern = r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}'

      df = df[~df['description'].str.contains(timestamp_pattern, na=False)]

      # Reset the DataFrame index
      df.reset_index(drop=True, inplace=True)

      columns_to_remove = ['id', 'raw_jobs_crawl_id', 'title', 'employer_description', 'employer_description', 'insertion_date', 'occupation']
      df = df.drop(columns=columns_to_remove)

      # Load the pre-trained model
      model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_MiniLM-L6')

      # Choose a subset of rows to compare the job descriptions
      values= 50;
      #values= 100;
      #values= 500;
      subset_df = df.head(values)
      sentences = subset_df['description'].tolist()

      # Generate HTML content for the table
      table_html = f'''
      <div id="table-tab" class="tab-content">
        <h2>Job Descriptions </h2>
        <table style="border-collapse: collapse; width: 100%; border: 1px solid #ddd;">
            <thead>
                <tr style="border: 1px solid #ddd; background-color: #f2f2f2;">
                <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Description Number</th>
                <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Description Details</th>
            </tr>
            </thead>
            <tbody>
    '''

      # Python code to insert descriptions into the HTML
      for index, description in enumerate(sentences):
          table_html += '''
                <tr style="border: 1px solid #ddd;">
                    <td style="padding: 8px; text-align: left; border: 1px solid #ddd;">Description {}</td>
                    <td style="padding: 8px; text-align: left; border: 1px solid #ddd;">{}</td>
                </tr>
        '''.format(index + 1, description)

      # Encode the sentences
      embeddings = model.encode(sentences)

      # Compute cosine similarity between all pairs
      cos_sim = util.cos_sim(embeddings, embeddings)

      # Create a graph
      G = nx.Graph()
      for i, _ in enumerate(sentences):
          G.add_node(i, label=f"Description {i+1}")
      for i in range(len(sentences)):
          for j in range(i + 1, len(sentences)):
              similarity = cos_sim[i][j]
              G.add_edge(i, j, weight=similarity)


      fig, ax2 = plt.subplots(1, figsize=(values, values))

      annot_font = {'size': 10, 'weight': 'bold'}
      # Plot the cosine similarity matrix
      ax2.set_title("Similarity Matrix For Jobs")
      sns.heatmap(cos_sim, annot=True, xticklabels=[f"Description {i+1}" for i in range(len(sentences))],yticklabels=[f"Description {i+1}" for i in range(len(sentences))], cmap="YlGnBu", ax=ax2, annot_kws=annot_font)

      # Perform hierarchical clustering
      linkage_matrix = linkage(cos_sim, method='ward')

      # Save plot to a buffer
      buffer = io.BytesIO()
      plt.savefig(buffer, format='png')
      buffer.seek(0)

      # Encode plot to base64 for HTML display
      plot_data = base64.b64encode(buffer.getvalue()).decode('utf-8')
      plt.close()

      javascript_code = '''
<script>
function openTab(evt, tabName) {
        var i, tabcontent, tablinks;
        tabcontent = document.getElementsByClassName("tab-content");
        for (i = 0; i < tabcontent.length; i++) {
            tabcontent[i].style.display = "none";
        }
        tablinks = document.getElementsByTagName("li");
        for (i = 0; i < tablinks.length; i++) {
            tablinks[i].className = "";
        }
        document.getElementById(tabName).style.display = "block";
        evt.currentTarget.className = "active";
    }

    function zoomIn() {
        var img = document.getElementById('graph-image');
        var currWidth = img.clientWidth;
        img.style.width = (currWidth + 100) + 'px';
    }

    function zoomOut() {
        var img = document.getElementById('graph-image');
        var currWidth = img.clientWidth;
        if (currWidth > 100) {
            img.style.width = (currWidth - 100) + 'px';
        }
    }

</script>
'''


      # HTML to display the plot, matrix heatmap, and search results in the browser
      html_content = f'''
      <!DOCTYPE html>
      <html>
      <html>
  <head>
      <title>Text Similarity Visualization</title>
      <style>
        /* Your CSS styles for tabs and content */
        body {{
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f4f4f4;
        }}
        .tabs ul {{
            list-style-type: none;
            padding: 0;
            margin: 0;
        }}
        .tabs ul li {{
            display: inline-block;
            margin-right: 10px;
            cursor: pointer;
            padding: 10px;
            background-color: #ddd;
            border-radius: 5px 5px 0 0;
        }}
        .tabs ul li.active {{
            font-weight: bold;
            background-color: #f4f4f4;
        }}
        .tab-content {{
            display: none;
            padding: 20px;
            border: 1px solid #ccc;
            border-top: none;
            background-color: #fff;
            border-radius: 0 0 5px 5px;
        }}
        .tab-content.active {{
            display: block;


        /* Additional styles for graph display */
        .graph-tab-content {{
            text-align: center;
        }}
        .graph-tab-content img {{
            max-width: 80%;
            display: block;
            margin: 0 auto;
        }}
        .graph-tab-content button {{
            margin-top: 10px;
            padding: 8px 20px;
            border-radius: 5px;
            background-color: #007bff;
            color: white;
            border: none;
            cursor: pointer;
        }}
        .graph-tab-content button:hover {{
            background-color: #0056b3;
        }}


    </style>
  </head>
      <body>
      <div class="tabs">
    <ul>
        <li onclick="openTab(event, 'welcome-tab')" class="active">Welcome</li>
        <li onclick="openTab(event, 'graph-tab')">Graph</li>
        <li onclick="openTab(event, 'table-tab')">Table</li>

    </ul>
</div>
<div id="welcome-tab" class="tab-content active">

    <h2>We welcome you to our project </h2>

</div>

<div id="graph-tab" class="tab-content">
    <h2>Sentence Similarity Graph</h2>
    <img id="graph-image" src="data:image/png;base64, {plot_data}" alt="Cosine Similarity Graph"  style="float: left; margin-right: 20px;"/>

    <!-- Your graph content here -->
    <!-- Zoom buttons and JavaScript code -->
    <button onclick="zoomIn()" style="background-color: #4CAF50; /* Green */
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;">Zoom In</button>

<button onclick="zoomOut()" style="background-color: #008CBA; /* Blue */
  border: none;
  color: white;
  padding: 15px 32px;
  text-align: center;
  text-decoration: none;
  display: inline-block;
  font-size: 16px;
  margin: 4px 2px;
  cursor: pointer;">Zoom Out</button>

    {javascript_code}
</div>



{table_html}

      </body>
      </html>
      '''




      return html_content

  if __name__ == '__main__':
      app.run()
